In [1]:
import pandas as pd
import numpy as np
import os
import gc
import requests
from datetime import datetime
import pandas as pd
from datetime import timedelta
import requests
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import time
import ast
import json
import numpy as np 
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
import json
import random
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import warnings
warnings.filterwarnings('ignore')
import pickle
from sklearn.linear_model import LinearRegression
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Input, Embedding, Dense, Concatenate, Flatten, GlobalAveragePooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split


2024-10-25 18:10:55.937226: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-25 18:10:55.937338: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered


In [2]:
events = pd.read_csv('events.csv')
item_features = pd.read_csv('item_features.csv')
user_features = pd.read_csv('user_features.csv')

In [3]:
events = events.sort_values(by = 'timestamp')

In [4]:
X_input, y_target = [], []
for i in tqdm(events.user_id.unique()):
    if len(events[events.user_id == i].iloc[-20:-10].item_id) == 10:
        X_input.append(list(events[events.user_id == i].iloc[-20:-10].item_id.values))
    else:
        lst = list(events[events.user_id == i].iloc[-20:-10].item_id.values)
        while len(lst) != 10:
            lst.insert(0, 0)
        X_input.append(lst)
    y_target.append(list(events[events.user_id == i].iloc[-10:].item_id.values))

100%|██████████| 6040/6040 [00:13<00:00, 457.28it/s]


In [5]:
X_test = []
for i in tqdm(events.user_id.unique()):
    if len(events[events.user_id == i].iloc[-10:].item_id) == 10:
        X_test.append(list(events[events.user_id == i].iloc[-10:].item_id.values))
    else:
        lst = list(events[events.user_id == i].iloc[-10:].item_id.values)
        while len(lst) != 10:
            lst.insert(0, 0)
        X_test.append(lst)


100%|██████████| 6040/6040 [00:08<00:00, 696.44it/s]


In [6]:
X_input, y_target, X_test = np.array(X_input), np.array(y_target), np.array(X_test)
X_train, X_val, y_train, y_val = train_test_split(X_input, y_target, test_size=0.05, random_state=42)

In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LayerNormalization, Dropout, Embedding, MultiHeadAttention
from tensorflow.keras.models import Model

class BERT4RTB(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(BERT4RTB, self).__init__()
        self.att = tf.keras.layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = tf.keras.Sequential([
            tf.keras.layers.Dense(ff_dim, activation="relu"), 
            tf.keras.layers.Dense(embed_dim),
        ])
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

def create_bert4rec_model(num_features, embed_dim, num_heads, ff_dim, mask_rate=0.15):
    inputs = Input(shape=(num_features,))  

    embedding_layer = Dense(embed_dim, activation='relu')(inputs) 
    x = tf.expand_dims(embedding_layer, axis=1) 

    positional_embedding = tf.range(start=0, limit=num_features, delta=1)
    pos_embedding_layer = Embedding(input_dim=num_features, output_dim=embed_dim)
    pos_encoding = pos_embedding_layer(positional_embedding)
    x += tf.expand_dims(pos_encoding, axis=0)  

    mask = tf.random.uniform(shape=tf.shape(x), minval=0, maxval=1) < mask_rate
    x = tf.where(mask, 0.0, x)  

    transformer_block = BERT4RTB(embed_dim, num_heads, ff_dim)
    x = transformer_block(x) 

    x = tf.squeeze(x, axis=1)

    x = Dense(9000, activation='relu')(x)
    outputs = Dense(1, activation='sigmoid')(x)  
    
    model = Model(inputs=inputs, outputs=outputs)
    return model


In [8]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Embedding, GlobalAveragePooling1D, Flatten
from tensorflow.keras.models import Model
max_sequence_length = 20  
n_preds = 10   
num_users = 6040 
num_films = events.item_id.max()+1  
def create_film_recommendation_model(num_users, events.item_id.max()+1 , embed_dim, num_heads, ff_dim, sequence_length, n_preds):
    user_input = Input(shape=(1,), name="user_input")
    film_sequence_input = Input(shape=(sequence_length - n_preds,), name="film_sequence_input")

    user_embedding = Embedding(input_dim=num_users, output_dim=embed_dim, name="user_embedding")(user_input)
    film_embedding = Embedding(input_dim=events.item_id.max()+1 , output_dim=embed_dim, name="film_embedding")(film_sequence_input)
    
    positional_encoding = Embedding(input_dim=sequence_length, output_dim=embed_dim, name="positional_encoding")
    positions = tf.range(start=0, limit=(sequence_length - n_preds), delta=1)
    pos_encoding = positional_encoding(positions)
    film_embedding += pos_encoding  

    transformer_block = BERT4RTBBERT4RecTransformerBlock(embed_dim, num_heads, ff_dim)
    x = transformer_block(film_embedding)

    x = GlobalAveragePooling1D()(x)

    x = Dense(9000, activation='relu')(x)
    outputs = Dense(events.item_id.max()+1  * n_preds, activation='softmax')(x)  
    
    outputs = tf.reshape(outputs, (-1, n_preds, events.item_id.max()+1 ))
    
    model = Model(inputs=[user_input, film_sequence_input], outputs=outputs)
    return model

embed_dim = 8  
num_heads = 4   
ff_dim = 32     
model = create_film_recommendation_model(num_users, events.item_id.max()+1 , embed_dim, num_heads, ff_dim, 20, n_preds)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',  
    metrics='accuracy'
)


In [9]:
history = model.fit([X_train[:, 0], X_train], y_train, validation_data=([X_val[:, 0], X_val], y_val), epochs=5, batch_size=32)

Epoch 1/5
180/180 [==============================] - 188s 1s/step - loss: 7.7896 - accuracy: 0.0038 - val_loss: 7.5661 - val_accuracy: 0.0043
Epoch 2/5
180/180 [==============================] - 185s 1s/step - loss: 7.0753 - accuracy: 0.0064 - val_loss: 7.6007 - val_accuracy: 0.0036
Epoch 3/5
180/180 [==============================] - 222s 1s/step - loss: 6.5033 - accuracy: 0.0094 - val_loss: 8.0007 - val_accuracy: 0.0066
Epoch 4/5
180/180 [==============================] - 198s 1s/step - loss: 5.8109 - accuracy: 0.0182 - val_loss: 8.5797 - val_accuracy: 0.0060
Epoch 5/5
180/180 [==============================] - 180s 1s/step - loss: 5.1454 - accuracy: 0.0398 - val_loss: 9.3117 - val_accuracy: 0.0043


In [10]:
preds = model.predict([X_test[:, 0], X_test])

189/189 [==============================] - 19s 102ms/step


In [14]:
ss = pd.read_csv('submission_sample.csv')

In [16]:
ss.item_id.iloc[0]

'0 1 2 3 4 5 6 7 8 9'

In [24]:
subm = {'user_id':[], 'item_id':[]}
count = 0
for i in tqdm(events.user_id.unique()):
    subm['user_id'].append(i)
    res = ''
    for j in preds[count]:
        res+=str(j.argmax())
        res+=' '
    count += 1
    subm['item_id'].append(res[:-1])
    

100%|██████████| 6040/6040 [00:00<00:00, 44534.83it/s]


In [23]:
for i in subm:
    print(i, len(subm[i]))

user_id 12080
item_id 6040


In [27]:
subm = pd.DataFrame(subm)
subm = subm.sort_values(by = 'user_id').reset_index(drop=True)

In [29]:
subm.to_csv('the_greatest_submussion(no).csv', index = False)